<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/Plan_and_execute_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://python.langchain.com/docs/modules/agents/agent_types/plan_and_execute

In [2]:
!pip install -q langchain langchain_experimental openai python-dotenv duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [3]:
import shutil
import os
import openai

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# Mount env direcroty
envfile_mount = '/content/drive/MyDrive/Colab Notebooks/env/'

# get secret
secrets_file_path = os.path.join(envfile_mount, 'env.txt')
shutil.copyfile(secrets_file_path, ".env")

# load env
from dotenv import load_dotenv
load_dotenv()

# just in the case...
openai.api_key = os.getenv("OPENAI_API_KEY")

Mounted at /content/drive


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
# from langchain import SerpAPIWrappe
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [6]:
# search = SerpAPIWrapper()
# Search
search = DuckDuckGoSearchRun()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to search queries."
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions about math."
    ),
]

In [7]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [8]:
response = agent.run("Do a SWOT analysis of Toyota Motor Corporation.")



> Entering new PlanAndExecute chain...
steps=[Step(value='Gather information about Toyota Motor Corporation.'), Step(value='Identify the strengths of Toyota Motor Corporation.'), Step(value='Identify the weaknesses of Toyota Motor Corporation.'), Step(value='Identify the opportunities for Toyota Motor Corporation.'), Step(value='Identify the threats to Toyota Motor Corporation.'), Step(value='Analyze the gathered information to create a SWOT analysis.'), Step(value='Given the above steps taken, provide a SWOT analysis of Toyota Motor Corporation.\n\n')]

> Entering new AgentExecutor chain...
Action:
{
  "action": "Search",
  "action_input": "Toyota Motor Corporation"
}

> Finished chain.
*****

Step: Gather information about Toyota Motor Corporation.

Response: Action:
{
  "action": "Search",
  "action_input": "Toyota Motor Corporation"
}

> Entering new AgentExecutor chain...
Thought: To identify the strengths of Toyota Motor Corporation, I can search for information about the compa

In [9]:
print(response)

Action:
{
  "action": "Final Answer",
  "action_input": "Strengths:\n1. Wide range of car variants\n2. Strong brand legacy\n3. Large workforce\n\nWeaknesses:\n1. Hierarchical organizational structure\n2. Secrecy in organizational culture\n3. Effects of product recalls\n\nOpportunities:\n- Please conduct a more targeted search using keywords like 'business opportunities for Toyota Motor Corporation' or 'growth opportunities for Toyota Motor Corporation' to gather more relevant information.\n\nThreats:\n1. Intense competition\n2. Technological disruptions\n3. Supply chain disruptions\n4. Changing regulations\n5. Economic downturns"
}
